In [ ]:
import pandas as pd
import polars as pl
import numpy as np
import networkx as nx
import json
import time
import sys
sys.path.insert(1, '../../rtsvg')
from rtsvg import *
rt = RACETrack()

In [ ]:
df = pd.read_csv('llama3_8b_2014_vast.csv')
_broken_ = []
for _response_ in df['llama3_8b_response']:
    i = _response_.index('```')
    j = _response_.index('```', i+1)
    try:
        json.loads(_response_[i+3:j])
    except:
        _broken_.append(_response_[i+3:j])
    print('.',end='')
len(_broken_), len(df), len(set(df['article'])), df['llama3_8b_time'].sum()

In [ ]:
df = pd.read_csv('phi_small_2014_vast.csv')
_broken_ = []
for _response_ in df['phi_response']:
    i = _response_.index('json')
    if '```' in _response_[i+1:]: j = _response_.index('```', i+1)
    else:                         j = len(_response_)
    try:
        json.loads(_response_[i+4:j])
    except:
        _broken_.append(_response_[i+4:j])
    print('.',end='')
len(_broken_), len(df), len(set(df['article'])), df['phi_time'].sum()

In [ ]:
df = pd.read_csv('mistral_7b_2014_vast.csv')
_broken_, _ok_ = [], []
for _response_ in df['mistral_7b_response']:
    _just_json_ = _response_[_response_.index('{'):_response_.rindex('}')+1]
    try:
        _json_ = json.loads(_just_json_)
        _ok_.append(_json_)
    except:
        _broken_.append(_just_json_)
    print('.',end='')
len(_broken_), len(df), len(set(df['article'])), df['mistral_7b_time'].sum()

In [ ]:
_json_repr_ = rt.jsonRepr(_ok_)
df, relates, node_labels = _json_repr_.starPathGraphDataFrame()
df  = pl.from_pandas(df)
g   = rt.createNetworkXGraph(df, relates)
pos = rt.hyperTreeLayout(g)
# pos = nx.spring_layout(g)
rt.interactiveGraphLayout(df, {'relationships':relates, 'pos':pos, 'node_labels':node_labels, 'draw_labels':True, 'node_size':'small'}, w=1200, h=800)